# 필요한 라이브러리 import

In [58]:
import csv
import pandas as pd
import os
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
plt.rc('font',family='Malgun Gothic')
plt.rc('font',size=10)
from sklearn.metrics import mean_absolute_error
import pickle
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings(action='ignore')

In [59]:
with open('./models/공격수_ss.pickle','rb') as f:
    forward_ss = pickle.load(f)
    
with open('./models/수비수_ss.pickle','rb') as f:
    defender_ss = pickle.load(f)
    
with open('./models/미드필더_ss.pickle','rb') as f:
    midfielder_ss = pickle.load(f)
    
with open('./models/공격수_pca.pickle','rb') as f:
    forward_pca = pickle.load(f)
    
with open('./models/수비수_pca.pickle','rb') as f:
    defender_pca = pickle.load(f)    

with open('./models/미드필더_pca.pickle','rb') as f:
    midfielder_pca = pickle.load(f)
    
with open('./models/공격수_model.pickle','rb') as f:
    forward_model = pickle.load(f)
    
with open('./models/미드필더_model.pickle','rb') as f:
    midfielder_model = pickle.load(f)
    
with open('./models/수비수_model.pickle','rb') as f:
    defender_model = pickle.load(f)

premier_df = pd.read_csv('./datas/NEW_FM_DATA_MERGED/2021-2022/2021-2022-PremierLeague.csv')
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중 A/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,K Ps/90,태클 성공/90,드리블 성공/90,헤더 성공/90,Cr A/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [60]:
premier_df.rename(columns={'Cr C/A':'크로스 성공률','FA':'파울 획득','Int/90':'인터셉트/90','공중 A/90':'공중볼시도/90','K Ps/90':'키패스/90','Cr A/90':'크로스 시도/90'},inplace=True)
premier_df = premier_df.astype({'골':np.int64,'도움':np.int64,'파울 획득':np.int64,'태클 성공률':np.int64})
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중볼시도/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [61]:
forward_df = premier_df[premier_df['포지션']=='공격수']
midfielder_df = premier_df[premier_df['포지션']=='미드필더']
defender_df = premier_df[premier_df['포지션']=='수비수']

In [62]:
use_col = ['골','도움','크로스 성공률','파울 획득','반칙 수/90', '기회 창출/90', '경기당 유효 슈팅/90', '경기당 슈팅/90', '인터셉트/90',
       '달린 거리/90분', '공중볼시도/90', '태클 성공률', '경기당 패스 시도/90', '경기당 패스 성공/90',
       '키패스/90', '태클 성공/90', '드리블 성공/90', '헤더 성공/90', '크로스 시도/90','평균획득승점/경기']

In [63]:
forward_df.loc[:,use_col[:-1]] = forward_ss.transform(forward_df.loc[:,use_col[:-1]])
midfielder_df.loc[:,use_col[:-1]] = midfielder_ss.transform(midfielder_df.loc[:,use_col[:-1]])
defender_df.loc[:,use_col[:-1]] = defender_ss.transform(defender_df.loc[:,use_col[:-1]])

In [64]:
forward_df.loc[:,use_col[:-1]] = forward_pca.transform(forward_df.loc[:,use_col[:-1]])
midfielder_df.loc[:,use_col[:-1]] = midfielder_pca.transform(midfielder_df.loc[:,use_col[:-1]])
defender_df.loc[:,use_col[:-1]] = defender_pca.transform(defender_df.loc[:,use_col[:-1]])

In [65]:
forward_df.loc[:,use_col[-1]]=forward_model.predict(forward_df.loc[:,use_col[:-1]])
midfielder_df.loc[:,use_col[-1]]=midfielder_model.predict(midfielder_df.loc[:,use_col[:-1]])
defender_df.loc[:,use_col[-1]]=defender_model.predict(defender_df.loc[:,use_col[:-1]])

In [66]:
forward_min = min(forward_df['평균획득승점/경기'].tolist())
midfielder_min = min(midfielder_df['평균획득승점/경기'].tolist())
defender_min = min(defender_df['평균획득승점/경기'].tolist())

In [67]:
forward_df['평균획득승점/경기'] = forward_df['평균획득승점/경기'].map(lambda x : x-forward_min+1)
midfielder_df['평균획득승점/경기'] = midfielder_df['평균획득승점/경기'].map(lambda x : x-midfielder_min+1)
defender_df['평균획득승점/경기'] = defender_df['평균획득승점/경기'].map(lambda x : x-defender_min+1)

In [68]:
all_df = pd.concat([forward_df,midfielder_df,defender_df])
all_df

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중볼시도/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,1.029580,-0.191765,0,7,11,...,0.065793,-0.713096,0.085785,0.035830,0.116508,-0.010428,0.291732,0.301885,0.046450,1.180247
3,Milot Rashica,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,-3.617324,-0.138959,0,8,9,...,-0.905892,-1.266459,-0.319230,0.968402,0.211179,-0.330103,0.339551,0.344472,0.068866,1.265721
4,Christos Tzolis,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,-0.307845,-2.087460,0,7,7,...,-0.000531,0.448636,0.128777,0.160195,-0.314032,0.145894,0.088229,0.166498,0.120650,1.280282
11,Callum Wilson,2021-2022,공격수,뉴캐슬,잉글랜드 프리미어 리그,3.373397,0.346909,0,20,5,...,-0.246822,-0.300967,-0.277388,0.312659,0.759946,0.228488,-0.237908,-0.209841,0.167163,2.033255
14,Allan Saint-Maximin,2021-2022,공격수,뉴캐슬,잉글랜드 프리미어 리그,-2.523324,0.794901,0,22,6,...,0.657919,-1.310765,0.675606,-0.619859,-0.550243,-0.404145,0.024166,0.146093,0.037275,1.994438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,Japhet Tanganga,2021-2022,수비수,토트넘,잉글랜드 프리미어 리그,3.453112,-1.628559,0,18,8,...,-0.158645,0.393515,0.221558,0.586290,0.090621,0.338196,0.016689,0.064101,0.154106,1.972114
292,Davinson Sánchez,2021-2022,수비수,토트넘,잉글랜드 프리미어 리그,-1.229028,1.474621,0,17,9,...,0.394238,0.171823,0.000074,-0.464654,0.242468,0.105898,-0.152872,0.254630,-0.019095,3.126329
294,Eric Dier,2021-2022,수비수,토트넘,잉글랜드 프리미어 리그,0.808505,-0.154644,0,12,5,...,0.302790,-0.515614,-0.386957,0.716689,-0.466496,0.573868,-0.159115,0.169845,-0.059747,2.293308
295,Ben Davies,2021-2022,수비수,토트넘,잉글랜드 프리미어 리그,7.224929,-5.205536,0,11,3,...,1.469321,-1.092427,-1.038637,-0.831584,1.318761,-0.031734,-0.499731,-0.538898,0.419938,1.832054


In [69]:
premier_df['WAR'] = all_df.sort_index()['평균획득승점/경기']

In [77]:
premier_df.sort_values(by='WAR',ascending=False).reset_index(drop=True).head(30)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,WAR
0,Davinson Sánchez,2021-2022,수비수,토트넘,잉글랜드 프리미어 리그,0,1,0,17,9,...,57,59.65,56.65,0.17,0.68,0.48,10.72,0.09,2.00,3.126329
1,Rodri,2021-2022,수비수,맨시티,잉글랜드 프리미어 리그,3,3,0,24,10,...,73,64.34,60.11,0.79,2.43,0.23,5.82,0.11,1.82,2.981597
2,Douglas Luiz,2021-2022,수비수,아스톤 빌라,잉글랜드 프리미어 리그,5,1,0,8,3,...,62,59.83,52.67,1.12,1.33,0.50,3.54,0.75,1.42,2.848492
3,N'Golo Kanté,2021-2022,미드필더,첼시,잉글랜드 프리미어 리그,6,23,0,34,10,...,72,51.91,45.85,1.29,1.95,0.54,2.24,1.37,2.00,2.817344
4,Bukayo Saka,2021-2022,미드필더,아스날,잉글랜드 프리미어 리그,13,9,0,26,4,...,83,35.92,31.41,1.62,2.81,3.33,4.70,5.58,1.91,2.745477
5,Phil Foden,2021-2022,공격수,맨시티,잉글랜드 프리미어 리그,8,5,0,21,2,...,77,42.61,37.41,1.79,2.45,2.21,3.09,4.57,2.50,2.719583
6,Tanguy Ndombele,2021-2022,미드필더,토트넘,잉글랜드 프리미어 리그,1,5,0,26,14,...,84,52.74,48.49,1.11,2.61,0.80,2.89,0.57,1.92,2.718248
7,Andriy Yarmolenko,2021-2022,미드필더,웨스트 햄,잉글랜드 프리미어 리그,5,2,0,13,11,...,84,30.66,25.89,1.64,4.12,1.40,14.90,3.45,1.32,2.705422
8,Riyad Mahrez,2021-2022,공격수,맨시티,잉글랜드 프리미어 리그,7,7,0,20,3,...,73,36.95,32.03,2.17,2.15,2.58,3.03,5.97,2.42,2.652088
9,Hakim Ziyech,2021-2022,미드필더,첼시,잉글랜드 프리미어 리그,5,6,0,21,5,...,90,39.14,33.95,1.62,3.37,1.75,5.60,3.89,2.19,2.609899


In [75]:
premier_df.query(f'이름 in ["Phil Foden"]')[['이름','WAR']]

,이름,WAR
5,Phil Foden,2.719583


In [73]:
my_players = ["Mohammed Salisu","Davinson Sánchez","Antonio Rüdiger","Ken Sema"
              ,"Jamie Shackleton","Oliver Skipp","Luke Thomas","Francisco Trincão"
              ,"Kieran Trippier","Jamie Vardy","Callum Wilson","Andriy Yarmolenko"
              ,"Yves Bissouma","Douglas Luiz","Pascal Groß"]
my_df = premier_df.query(f'이름 in {my_players}')[['이름','WAR']]

display(my_df.sort_values(by="WAR",ascending=False).head(10))
print("WAR 평균",np.mean(my_df["WAR"]))

,이름,WAR
0,Davinson Sánchez,3.126329
2,Douglas Luiz,2.848492
7,Andriy Yarmolenko,2.705422
10,Mohammed Salisu,2.579638
11,Yves Bissouma,2.574214
12,Ken Sema,2.574142
15,Francisco Trincão,2.542016
16,Antonio Rüdiger,2.512160
48,Luke Thomas,2.263476
51,Kieran Trippier,2.257739


WAR 평균 2.398552004086361


In [72]:
cpfc_players = ["Nathaniel Clyne","Odsonne Edouard","Ebere Eze","Marc Guehi","Will Hughes",
               "Cheikhou Kouyaté","Luka Milivojević","Tyrick Mitchell","Michael Olise","Jairo Riedewald",
               "Wilfried Zaha","Joachim Andersen"]


cpfc_df = premier_df.query(f'이름 in {cpfc_players}')[['이름','WAR']].sort_values(by="WAR",ascending=False)

display(cpfc_df.sort_values(by="WAR",ascending=False).head(10))
print("WAR 평균",np.mean(cpfc_df["WAR"]))

,이름,WAR
73,Cheikhou Kouyaté,2.146257
167,Tyrick Mitchell,1.782043
192,Michael Olise,1.704212
201,Ebere Eze,1.647425
203,Marc Guehi,1.638107
219,Wilfried Zaha,1.561346
221,Odsonne Edouard,1.547845
224,Luka Milivojević,1.543938
229,Joachim Andersen,1.513269
231,Nathaniel Clyne,1.508818


WAR 평균 1.622769563322563
